<a href="https://colab.research.google.com/github/ShadyH20/NLP-project/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading

In [203]:
!pip install farasa
!pip install farasapy
!pip install arabic-reshaper
!pip install python-bidi
!pip install qalsadi
!pip install langdetect
!pip install googletrans


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to us

In [204]:
import os
import pandas as pd
import numpy as np
import json
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from itertools import islice
from nltk.util import ngrams
import arabic_reshaper
from bidi.algorithm import get_display
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.isri import ISRIStemmer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from qalsadi.lemmatizer import Lemmatizer
from langdetect import detect
from googletrans import Translator
from tashaphyne.stemming import ArabicLightStemmer
from farasa.diacratizer import FarasaDiacritizer
from qalsadi.lemmatizer import Lemmatizer

import nltk
import spacy
nlp = spacy.load('en_core_web_sm')
from nltk import tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package 

True

### Unlabeled Data (Da7ee7, Elsaha)

In [205]:
BASE_DIR = "./YoutubeChannels"
CHANNELS = ["Da7ee7", "Elsaha"]
data = []

timestamp_pattern = re.compile(r"^\d+\.\d+:\s*")  # Removes timestamps at start of lines

for channel in CHANNELS:
    raw_data_path = os.path.join(BASE_DIR, channel, "Raw Data")
    
    if os.path.exists(raw_data_path):
        for file_name in os.listdir(raw_data_path):
            if file_name.endswith(".txt"):
                file_path = os.path.join(raw_data_path, file_name)
                
                with open(file_path, "r", encoding="utf-8") as f:
                    raw_lines = f.readlines()
                
                # Remove timestamps and extra whitespace from lines
                cleaned_text = "\n".join(re.sub(timestamp_pattern, "", line).strip() for line in raw_lines)
                episode_title = file_name.replace(".txt", "").strip()
                
                # Append the entire script as one row
                data.append([channel, episode_title, cleaned_text, "", ""])

# Create DataFrame
unlabelled_df = pd.DataFrame(data, columns=["Channel", "EpisodeTitle", "Original Script", "Processed Script", "Category"])

### Labeled Data

In [ ]:
BASE_DIR = "./YoutubeChannels"
LABELLED_CHANNELS_1 = ["Kefaya Ba2a", "Business Bel Araby", "B Hodoo2"]
LABELLED_CHANNELS_2 = ["Eqtisad_Al_Hadaraa", "Fi_Al_Hadaraa", "Al_Mokhbir_Al_Eqtisadi"]
LABELLED_CHANNELS_3 = ["Akhdar"]
data = []

def process_labelled_channel_1(channel):
    raw_data_path = os.path.join(BASE_DIR, channel, "raw")
    annotations_path = os.path.join(BASE_DIR, channel, "annotations.json")
    
    if os.path.exists(annotations_path):
        with open(annotations_path, "r", encoding="utf-8") as f:
            annotations = json.load(f)
        
        metadata_map = {
            item["title"].strip()[:10] if channel == "B Hodoo2" else item["title"].strip(): item
            for item in annotations
        }
        
        if os.path.exists(raw_data_path):
            for file_name in os.listdir(raw_data_path):
                if file_name.endswith(".txt"):
                    file_path = os.path.join(raw_data_path, file_name)
                    with open(file_path, "r", encoding="utf-8") as f:
                        raw_text = f.read().strip()
                    
                    episode_title = file_name.replace(".txt", "").strip()
                    match_key = episode_title[:10] if channel == "B Hodoo2" else episode_title
                    metadata = metadata_map.get(match_key, {})
                    
                    data.append([
                        channel,
                        episode_title,
                        raw_text,
                        "",
                        metadata.get("dialogue", ""),
                        metadata.get("type", ""),
                        metadata.get("length", ""),
                        metadata.get("category", "")
                    ])

def process_labelled_channel_2(channel):
    raw_data_path = os.path.join(BASE_DIR, channel, "raw_data")
    metadata_path = os.path.join(BASE_DIR, channel, "metadata")
    
    if os.path.exists(raw_data_path) and os.path.exists(metadata_path):
        for file_name in os.listdir(raw_data_path):
            if file_name.endswith(".txt"):
                episode_title = file_name.replace(".txt", "").strip()
                raw_file_path = os.path.join(raw_data_path, file_name)
                metadata_file_path = os.path.join(metadata_path, episode_title + ".json")
                
                with open(raw_file_path, "r", encoding="utf-8") as f:
                    raw_text = f.read().strip()
                
                if os.path.exists(metadata_file_path):
                    with open(metadata_file_path, "r", encoding="utf-8") as f:
                        metadata = json.load(f)
                    category = metadata.get("categories", [""])[0]
                    
                    data.append([
                        channel,
                        episode_title,
                        raw_text,
                        "",
                        metadata.get("dialogue", ""),
                        metadata.get("type", ""),
                        metadata.get("length", ""),
                        category
                    ])

def find_akhdar_raw_and_metadata(path):
    for root, dirs, files in os.walk(path):
        if "raw_data" in dirs and "metadata" in dirs:
            process_labelled_channel_2(root)

def process_labelled_channel_3(channel):
    base_path = os.path.join(BASE_DIR, channel)
    find_akhdar_raw_and_metadata(base_path)

# Process all channels
for channel in LABELLED_CHANNELS_1:
    process_labelled_channel_1(channel)

for channel in LABELLED_CHANNELS_2:
    process_labelled_channel_2(channel)

for channel in LABELLED_CHANNELS_3:
    process_labelled_channel_3(channel)

labelled_df = pd.DataFrame(data, columns=["Channel", "Episode Title", "Original Script", "Processed Script", "Dialogue", "Type", "Length", "Category"])

### Data Loading Results

In [207]:
def sample_rows_per_channel(df, channel_column="Channel", num_samples=3):
    unique_channels = df[channel_column].unique()
    for channel in unique_channels:
        print(f"📌 Sample rows from channel: {channel}")
        display(df[df[channel_column] == channel].sample(min(num_samples, len(df[df[channel_column] == channel]))))
        print("-" * 80)

print("📍 Checking Unlabelled DataFrame:")
sample_rows_per_channel(unlabelled_df)

print("\n📍 Checking Labelled DataFrame:")
sample_rows_per_channel(labelled_df)


📍 Checking Unlabelled DataFrame:
📌 Sample rows from channel: Da7ee7


,Channel,EpisodeTitle,Original Script,Processed Script,Category
54,Da7ee7,نابليون في القاهرة الدحيح,يا بوي! دا أني تعبت جوي!\n!Bonjour\nأفندم؟!\nP...,,
6,Da7ee7,هانيبال الدحيح,"أوه، ""ماركينوس""، لم أعُد أستطيع النوم،\nمن فرط...",,
1,Da7ee7,تاريخ التشريح الدحيح,"أنا الدكتور ""ياسر الطائي""،\nأكتب الآن تقرير تش...",,


--------------------------------------------------------------------------------
📌 Sample rows from channel: Elsaha


,Channel,EpisodeTitle,Original Script,Processed Script,Category
163,Elsaha,أقوى من الخوف والإعاقة.. حكاية عبد الرحمن صلاح,[موسيقى]\nاول ما لبست الطرف طبعا كان مضايقني ب...,,
159,Elsaha,ليست دينية مسيحية.. حكاية جريدة وطني,وطني صدرت\n1958 وانا كان سني ثمان سنين والدي ا...,,
154,Elsaha,واتر البحري.. مصور حياة طبيعية,اي مصور حياه بريه يقول لك لو انت رايح\nتصور طي...,,


--------------------------------------------------------------------------------

📍 Checking Labelled DataFrame:
📌 Sample rows from channel: Kefaya Ba2a


...

--------------------------------------------------------------------------------
📌 Sample rows from channel: Fi_Al_Hadaraa


,Channel,Episode Title,Original Script,Processed Script,Dialogue,Type,Length,Category
94,Fi_Al_Hadaraa,هالاند_تريبل_كابتن__في_الحضارة,يعني خلاص؟\nما فيش منّه تاني؟\nما خلاص بقى يا ...,,,Youtube,00:24:18,Entertainment
85,Fi_Al_Hadaraa,لماذا_نحتاج_التعلم_بشكل_مستمر_للحفاظ_على_الوظيفة,دلوقتي المؤسسات والشركات بقت مخليها\nمساله تدر...,,,Youtube,00:00:58,People & Blogs
73,Fi_Al_Hadaraa,اللغة__كيف_فقدنا_الحساسية_للغة_الجميلة__في_الح...,يا نجف بنّور، صديقي الإنسان. صديقي الإنسان!\nا...,,,Youtube,00:17:23,People & Blogs


--------------------------------------------------------------------------------
📌 Sample rows from channel: Al_Mokhbir_Al_Eqtisadi


,Channel,Episode Title,Original Script,Processed Script,Dialogue,Type,Length,Category
114,Al_Mokhbir_Al_Eqtisadi,المخبر_الاقتصادي__كيف_خططت_أمريكا_سرا_لتعقيم_س...,في أوائل التسعينيات الرأي العام في البرازيل\nك...,,,Youtube,00:14:21,Entertainment
175,Al_Mokhbir_Al_Eqtisadi,المخبر_الاقتصادي__كيف_دفع_رجل_وامرأة_اقتصاد_بر...,راجل وواحدة ست\nالاتنين المحترمين دول طلعوا في...,,,Youtube,00:21:00,Entertainment
315,Al_Mokhbir_Al_Eqtisadi,المخبر_الاقتصادي__كيف_أصبح_المغرب_حارس_بوابة_ا...,عارفين هولندا؟\nأكيد عارفينها\nوأكيد برضو سمعت...,,,Youtube,00:14:30,Entertainment


--------------------------------------------------------------------------------


# Analysis & Processing

### 2. Analysis on Original Script
	
	Channel Analysis:
	- avg length / duration of episodes per channel
	- modal categories per channel, Category / class imbalance
	- etc
	- metdata analysis
	- Text Charactertistics (formal / informal... dialogue?)
	
	NLP Analysis:
	- Text Sampling
	- Text Statistics (word count, sentence length...)

In [ ]:
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

def arabic_tokenizer(text):
    return tokenize.word_tokenize(text)

# Combine Arabic + English stopwords
ALL_STOPWORDS = set(stopwords.words("arabic") + stopwords.words("english"))

def analyze_channel_data(labelled_df):
    print("=== CHANNEL ANALYSIS ===")

    df = labelled_df.copy()
    df = df.dropna(subset=["Category"])
    df["Script Length"] = df["Original Script"].apply(lambda x: len(arabic_tokenizer(x)))

    # Average Script length per channel
    avg_length = df.groupby("Channel")["Script Length"].mean()
    print("\n📌 Average Episode Length per Channel:\n", avg_length)

    # Number of episodes per channel
    episode_count = df.groupby("Channel")["Episode Title"].nunique()
    print("\n📌 Number of Episodes per Channel:\n", episode_count)

    # Most common category per channel
    modal_category = df.groupby("Channel")["Category"].agg(lambda x: x.value_counts().idxmax() if x.value_counts().size > 0 else "Unknown")
    print("\n📌 Most Common Category per Channel:\n", modal_category)

    # Plot category distribution
    plt.figure(figsize=(12, 6))
    category_distribution = df["Category"].value_counts()
    sns.barplot(x=category_distribution.index, y=category_distribution.values, palette="viridis")
    plt.xticks(rotation=45)
    plt.title("Category Distribution")
    plt.show()

    # Dialogue analysis
    if "Dialogue" in df.columns and df["Dialogue"].notna().sum() > 0:
        dialogue_counts = df["Dialogue"].dropna().value_counts()
        plt.figure(figsize=(6, 4))
        sns.barplot(x=dialogue_counts.index, y=dialogue_counts.values, palette="coolwarm")
        plt.title("Dialogue vs. Non-Dialogue Distribution")
        plt.show()

def analyze_nlp_statistics(labelled_df):
    print("=== NLP ANALYSIS ===")

    df = labelled_df.copy()
    df["Word Count"] = df["Original Script"].apply(lambda x: len(arabic_tokenizer(x)))
    print("\n📌 Word Count Statistics:\n", df["Word Count"].describe())

    # Lexical diversity
    all_words = [word.lower() for sentence in df["Original Script"] for word in arabic_tokenizer(sentence)]
    unique_words = set(all_words)
    lexical_diversity = len(unique_words) / len(all_words) if all_words else 0
    print("\n📌 Lexical Diversity:", round(lexical_diversity, 4))

    # Stopword ratio
    filtered_words = [word for word in all_words if word not in ALL_STOPWORDS]
    stopword_ratio = 1 - (len(filtered_words) / len(all_words)) if all_words else 0
    print("\n📌 Percentage of Stopwords:", round(stopword_ratio * 100, 2), "%")

    # Most common words
    common_words = Counter(filtered_words)
    print("\n📌 Most Common Words:\n", common_words.most_common(10))

    # Common bigrams and trigrams
    bigrams = list(ngrams(filtered_words, 2))
    trigrams = list(ngrams(filtered_words, 3))
    print("\n📌 Most Common Bigrams:\n", Counter(bigrams).most_common(10))
    print("\n📌 Most Common Trigrams:\n", Counter(trigrams).most_common(10))

Abbreviations Analysis

In [ ]:
import re
from collections import Counter

def extract_abbreviations_per_channel(df, text_column="Original Script", channel_column="Channel"):
    # Match words with only capital letters, min length 2
    abbreviation_pattern = re.compile(r"\b[A-Z]{2,}\b")
    
    channel_abbreviation_stats = {}

    for channel, group in df.groupby(channel_column):
        all_abbreviations = []
        
        for text in group[text_column].dropna():
            abbreviations = abbreviation_pattern.findall(text)
            all_abbreviations.extend(abbreviations)
        
        abbreviation_counts = Counter(all_abbreviations)
        channel_abbreviation_stats[channel] = abbreviation_counts
    
    return channel_abbreviation_stats

# Run on both labelled and unlabelled datasets
print("=== Labelled Dataset ===")
labelled_abbreviations = extract_abbreviations_per_channel(labelled_df)
for channel, counts in labelled_abbreviations.items():
    print(f"\n📌 Channel: {channel}")
    print(f"Total Unique Abbreviations: {len(counts)}")
    print("Abbreviations & Counts:")
    for abbr, count in counts.items():
        print(f"{abbr}: {count} times")

print("\n=== Unlabelled Dataset ===")
unlabelled_abbreviations = extract_abbreviations_per_channel(unlabelled_df)
for channel, counts in unlabelled_abbreviations.items():
    print(f"\n📌 Channel: {channel}")
    print(f"Total Unique Abbreviations: {len(counts)}")
    print("Abbreviations & Counts:")
    for abbr, count in counts.items():
        print(f"{abbr}: {count} times")

=== Labelled Dataset ===

📌 Channel: Al_Mokhbir_Al_Eqtisadi
Total Unique Abbreviations: 91
Abbreviations & Counts:
CIA: 1 times
USAID: 1 times
FOMC: 5 times
IOS: 1 times
CFA: 22 times
SDECE: 1 times
GSM: 2 times
ETLA: 2 times
BASF: 1 times
ISO: 3 times
AEPR: 3 times
AK: 1 times
NNPC: 1 times
DGB: 1 times
AG: 3 times
GMBH: 1 times
FBI: 1 times
IQ: 2 times
FOMO: 1 times
SE: 1 times
SAMA: 1 times
SDR: 3 times
ARAMCO: 1 times
LNG: 1 times
BIS: 1 times
NUDT: 1 times
EDA: 4 times
ASML: 61 times
FDPR: 1 times
SMIC: 2 times
DRAM: 1 times
NAND: 2 times
CCD: 1 times
MH: 1 times
SES: 1 times
MSNBC: 1 times
ABS: 1 times
CEF: 1 times
AA: 1 times
OPC: 10 times
DUV: 6 times
EUV: 13 times
SK: 1 times
CFIUS: 3 times
DOI: 2 times
GIC: 3 times
ITRI: 1 times
TSMC: 5 times
CSET: 1 times
API: 2 times
KPMG: 1 times
CII: 1 times
BYD: 64 times
PHEV: 1 times
NA: 1 times
NBER: 1 times
FTX: 63 times
US: 2 times
BUSD: 1 times
FTT: 21 times
AIG: 1 times
CRP: 2 times
ATM: 1 times
CNC: 1 times
NDTV: 1 times
ERM: 2 ti

### 3. Preprocessing
	- Punctuation, Symbols, Numbers, Diactritics (tashkeel) removal
	- Stemming / lemmatization?
	- Normalization of Arabic letters
	- Code switching handling? Translation? What about abbreviations?

import nltk
nltk.download('punkt')
import re
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Arabic normalization map
ARABIC_NORMALIZATION_MAP = str.maketrans({
    "أ": "ا",
    "إ": "ا",
    "آ": "ا",
    "ى": "ي",
    "ة": "ه",
})

# Arabic punctuation characters
ARABIC_PUNCTUATION = "؟،؛«»ـ…“”‘’–—"

# Combined punctuation
ALL_PUNCTUATION = string.punctuation + ARABIC_PUNCTUATION

# Arabic Stopword list (can be extended)
ARABIC_STOPWORDS = set([
    "في", "من", "على", "ما", "و", "لا", "عن", "إلى", "أن", "هو", "هي", "هذا", "ذلك", "كل", "كان", "كما",
    "لذلك", "أو", "أي", "لم", "قد", "أكثر", "أقل", "هنا", "هناك", "بعد", "قبل", "بين", "مع", "حتى", "إذا",
    "ثم", "لكن", "أحد", "أيضا", "أثناء", "عند", "أين", "كيف", "إلا", "أصبح", "لأن", "بسبب", "هذه", "هؤلاء",
    "التي", "الذي", "الذين", "اللذي", "اللذين", "اللائي"
])

# English abbreviations dictionary
ENGLISH_ABBREVIATIONS = {
    "CIA": "وكالة المخابرات المركزية",
    "USAID": "الوكالة الأمريكية للتنمية الدولية",
    "FOMC": "اللجنة الفيدرالية للسوق المفتوحة",
    "IOS": "نظام تشغيل آي أو إس",
    "CFA": "محلل مالي معتمد",
    "SDECE": "الخدمة السرية للدفاع الوطني",
    "GSM": "النظام العالمي للاتصالات المتنقلة",
    "ETLA": "الاتحاد الأوروبي للتكنولوجيا والتعليم",
    "BASF": "شركة الكيميائيات الألمانية",
    "ISO": "المنظمة الدولية للتوحيد القياسي",
    "AEPR": "التقييم السنوي للأداء الاقتصادي",
    "AK": "كلاشينكوف",
    "NNPC": "شركة البترول الوطنية النيجيرية",
    "DGB": "الإدارة العامة للأمن",
    "AG": "شركة ذات مسؤولية محدودة",
    "GMBH": "شركة محدودة المسؤولية في ألمانيا",
    "FBI": "مكتب التحقيقات الفيدرالي",
    "IQ": "معدل الذكاء",
    "FOMO": "الخوف من تفويت الفرصة",
    "SE": "هندسة البرمجيات",
    "SAMA": "مؤسسة النقد العربي السعودي",
    "SDR": "حقوق السحب الخاصة",
    "ARAMCO": "شركة أرامكو السعودية",
    "LNG": "الغاز الطبيعي المسال",
    "BIS": "بنك التسويات الدولية",
    "NUDT": "جامعة الدفاع الوطني للتكنولوجيا",
    "EDA": "إدارة التنمية الاقتصادية",
    "ASML": "شركة تكنولوجيا أشباه الموصلات",
    "FDPR": "لوائح المنتجات الأجنبية المباشرة",
    "SMIC": "شركة تصنيع أشباه الموصلات الصينية",
    "DRAM": "ذاكرة الوصول العشوائي الديناميكية",
    "NAND": "ذاكرة فلاش ناند",
    "CCD": "جهاز الشحن المزدوج",
    "MH": "ميغاهيرتز",
    "SES": "أنظمة الاتصالات الفضائية",
    "MSNBC": "قناة الأخبار الأمريكية",
    "ABS": "نظام الفرامل المانع للانغلاق",
    "CEF": "صندوق الصرف الأوروبي",
    "AA": "الخطوط الجوية الأمريكية",
    "OPC": "لجنة العمليات",
    "DUV": "الأشعة فوق البنفسجية العميقة",
    "EUV": "الأشعة فوق البنفسجية الشديدة",
    "CFIUS": "لجنة الاستثمار الأجنبي في الولايات المتحدة",
    "DOI": "وزارة الداخلية",
    "GIC": "شركة الاستثمار الحكومية",
    "ITRI": "معهد الأبحاث الصناعية",
    "TSMC": "شركة تصنيع أشباه الموصلات التايوانية",
    "API": "واجهة برمجة التطبيقات",
    "KPMG": "شركة التدقيق العالمية",
    "CII": "اتحاد الصناعات الهندية",
    "BYD": "شركة السيارات الكهربائية الصينية",
    "PHEV": "السيارات الكهربائية الهجينة",
    "NBER": "المكتب الوطني للبحوث الاقتصادية",
    "FTX": "منصة تداول العملات الرقمية",
    "US": "الولايات المتحدة",
    "BUSD": "عملة بينانس المستقرة",
    "FTT": "رمز FTX الأصلي",
    "AIG": "مجموعة التأمين الأمريكية",
    "CRP": "برنامج الحفظ الزراعي",
    "ATM": "الصراف الآلي",
    "CNC": "التحكم الرقمي بالكمبيوتر",
    "NDTV": "القناة الهندية الإخبارية",
    "ERM": "إدارة المخاطر المؤسسية",
    "YCC": "التحكم في منحنى العائد",
    "ENN": "شركة الطاقة الصينية",
    "ECO": "النظام البيئي",
    "SEC": "لجنة الأوراق المالية والبورصات",
    "CHIPS": "قانون دعم إنتاج أشباه الموصلات",
    "CCIEE": "المركز الصيني للتبادلات الاقتصادية الدولية",
    "WS": "وول ستريت",
    "MD": "المدير الطبي",
    "FAA": "إدارة الطيران الفيدرالية",
    "IATA": "الاتحاد الدولي للنقل الجوي",
    "MIT": "معهد ماساتشوستس للتكنولوجيا",
    "PACS": "نظام أرشفة الصور والاتصالات",
    "USB": "منفذ الناقل التسلسلي العالمي",
    "NASA": "وكالة الفضاء الأمريكية",
    "WWE": "المصارعة العالمية الترفيهية",
    "GPS": "نظام تحديد المواقع العالمي"
}

import qalsadi.lemmatizer
import pyarabic.araby as araby

# Initialize Qalsadi lemmatizer
lemmatizer = qalsadi.lemmatizer.Lemmatizer()

def preprocess_text(text):
    if not isinstance(text, str) or not text.strip():
        return ""

    # Replace newline characters with space
    text = text.replace("\n", " ")

    # Normalize Arabic letters
    text = text.translate(ARABIC_NORMALIZATION_MAP)

    # Replace English abbreviations
    words = text.split()
    replaced_words = []
    for word in words:
        if re.match(r"^[A-Z]{2,}$", word):
            word = ENGLISH_ABBREVIATIONS.get(word.upper(), word)
        replaced_words.append(word)
    text = " ".join(replaced_words)

    # Remove punctuation
    text = re.sub(f"[{re.escape(ALL_PUNCTUATION)}]", " ", text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    filtered_tokens = [tok for tok in tokens if tok not in ARABIC_STOPWORDS]

    # Apply lemmatization using Qalsadi
    lemmatized_tokens = [lemmatizer.lemmatize(tok) for tok in filtered_tokens]

    return " ".join(lemmatized_tokens)

# Apply preprocessing
for df in [unlabelled_df, labelled_df]:
    df["Processed Script"] = df["Original Script"].apply(preprocess_text)

print("✅ Preprocessing completed (NLTK tokenization, stopwords, punctuation, newline handled).")

In [ ]:
import nltk
nltk.download('punkt')
import re
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')

ARABIC_NORMALIZATION_MAP = str.maketrans({
    "أ": "ا",
    "إ": "ا",
    "آ": "ا",
    "ى": "ي",
    "ة": "ه",
})

ARABIC_PUNCTUATION = "؟،؛«»ـ…“”‘’–—"

ALL_PUNCTUATION = string.punctuation + ARABIC_PUNCTUATION

ARABIC_STOPWORDS = set([
    "في", "من", "على", "ما", "و", "لا", "عن", "إلى", "أن", "هو", "هي", "هذا", "ذلك", "كل", "كان", "كما",
    "لذلك", "أو", "أي", "لم", "قد", "أكثر", "أقل", "هنا", "هناك", "بعد", "قبل", "بين", "مع", "حتى", "إذا",
    "ثم", "لكن", "أحد", "أيضا", "أثناء", "عند", "أين", "كيف", "إلا", "أصبح", "لأن", "بسبب", "هذه", "هؤلاء",
    "التي", "الذي", "الذين", "اللذي", "اللذين", "اللائي"
])

ENGLISH_ABBREVIATIONS = {
    "CIA": "وكالة المخابرات المركزية",
    "USAID": "الوكالة الأمريكية للتنمية الدولية",
    "FOMC": "اللجنة الفيدرالية للسوق المفتوحة",
    "IOS": "نظام تشغيل آي أو إس",
    "CFA": "محلل مالي معتمد",
    "SDECE": "الخدمة السرية للدفاع الوطني",
    "GSM": "النظام العالمي للاتصالات المتنقلة",
    "ETLA": "الاتحاد الأوروبي للتكنولوجيا والتعليم",
    "BASF": "شركة الكيميائيات الألمانية",
    "ISO": "المنظمة الدولية للتوحيد القياسي",
    "AEPR": "التقييم السنوي للأداء الاقتصادي",
    "AK": "كلاشينكوف",
    "NNPC": "شركة البترول الوطنية النيجيرية",
    "DGB": "الإدارة العامة للأمن",
    "AG": "شركة ذات مسؤولية محدودة",
    "GMBH": "شركة محدودة المسؤولية في ألمانيا",
    "FBI": "مكتب التحقيقات الفيدرالي",
    "IQ": "معدل الذكاء",
    "FOMO": "الخوف من تفويت الفرصة",
    "SE": "هندسة البرمجيات",
    "SAMA": "مؤسسة النقد العربي السعودي",
    "SDR": "حقوق السحب الخاصة",
    "ARAMCO": "شركة أرامكو السعودية",
    "LNG": "الغاز الطبيعي المسال",
    "BIS": "بنك التسويات الدولية",
    "NUDT": "جامعة الدفاع الوطني للتكنولوجيا",
    "EDA": "إدارة التنمية الاقتصادية",
    "ASML": "شركة تكنولوجيا أشباه الموصلات",
    "FDPR": "لوائح المنتجات الأجنبية المباشرة",
    "SMIC": "شركة تصنيع أشباه الموصلات الصينية",
    "DRAM": "ذاكرة الوصول العشوائي الديناميكية",
    "NAND": "ذاكرة فلاش ناند",
    "CCD": "جهاز الشحن المزدوج",
    "MH": "ميغاهيرتز",
    "SES": "أنظمة الاتصالات الفضائية",
    "MSNBC": "قناة الأخبار الأمريكية",
    "ABS": "نظام الفرامل المانع للانغلاق",
    "CEF": "صندوق الصرف الأوروبي",
    "AA": "الخطوط الجوية الأمريكية",
    "OPC": "لجنة العمليات",
    "DUV": "الأشعة فوق البنفسجية العميقة",
    "EUV": "الأشعة فوق البنفسجية الشديدة",
    "CFIUS": "لجنة الاستثمار الأجنبي في الولايات المتحدة",
    "DOI": "وزارة الداخلية",
    "GIC": "شركة الاستثمار الحكومية",
    "ITRI": "معهد الأبحاث الصناعية",
    "TSMC": "شركة تصنيع أشباه الموصلات التايوانية",
    "API": "واجهة برمجة التطبيقات",
    "KPMG": "شركة التدقيق العالمية",
    "CII": "اتحاد الصناعات الهندية",
    "BYD": "شركة السيارات الكهربائية الصينية",
    "PHEV": "السيارات الكهربائية الهجينة",
    "NBER": "المكتب الوطني للبحوث الاقتصادية",
    "FTX": "منصة تداول العملات الرقمية",
    "US": "الولايات المتحدة",
    "BUSD": "عملة بينانس المستقرة",
    "FTT": "رمز FTX الأصلي",
    "AIG": "مجموعة التأمين الأمريكية",
    "CRP": "برنامج الحفظ الزراعي",
    "ATM": "الصراف الآلي",
    "CNC": "التحكم الرقمي بالكمبيوتر",
    "NDTV": "القناة الهندية الإخبارية",
    "ERM": "إدارة المخاطر المؤسسية",
    "YCC": "التحكم في منحنى العائد",
    "ENN": "شركة الطاقة الصينية",
    "ECO": "النظام البيئي",
    "SEC": "لجنة الأوراق المالية والبورصات",
    "CHIPS": "قانون دعم إنتاج أشباه الموصلات",
    "CCIEE": "المركز الصيني للتبادلات الاقتصادية الدولية",
    "WS": "وول ستريت",
    "MD": "المدير الطبي",
    "FAA": "إدارة الطيران الفيدرالية",
    "IATA": "الاتحاد الدولي للنقل الجوي",
    "MIT": "معهد ماساتشوستس للتكنولوجيا",
    "PACS": "نظام أرشفة الصور والاتصالات",
    "USB": "منفذ الناقل التسلسلي العالمي",
    "NASA": "وكالة الفضاء الأمريكية",
    "WWE": "المصارعة العالمية الترفيهية",
    "GPS": "نظام تحديد المواقع العالمي"
}

def preprocess_text(text):
    if not isinstance(text, str) or not text.strip():
        return ""

    text = text.replace("\n", " ")

    # Normalize Arabic letters
    text = text.translate(ARABIC_NORMALIZATION_MAP)

    # Replace English abbreviations
    words = text.split()
    replaced_words = []
    for word in words:
        if re.match(r"^[A-Z]{2,}$", word):
            word = ENGLISH_ABBREVIATIONS.get(word.upper(), word)
        replaced_words.append(word)
    text = " ".join(replaced_words)

    # Remove punctuation
    text = re.sub(f"[{re.escape(ALL_PUNCTUATION)}]", " ", text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    filtered_tokens = [tok for tok in tokens if tok not in ARABIC_STOPWORDS]

    return " ".join(filtered_tokens)

# Apply preprocessing
for df in [unlabelled_df, labelled_df]:
    df["Processed Script"] = df["Original Script"].apply(preprocess_text)

print("✅ Preprocessing completed (NLTK tokenization, stopwords, punctuation, newline handled).")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sherifahammoud/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ Preprocessing completed (NLTK tokenization, stopwords, punctuation, newline handled).


### 4. Analysis (On Proc) 1
	- Text Sampling
	- Text Visualizations

In [211]:
def sample_rows_per_channel(df, channel_column="Channel", num_samples=3):
    unique_channels = df[channel_column].unique()
    for channel in unique_channels:
        print(f"📌 Sample rows from channel: {channel}")
        display(df[df[channel_column] == channel].sample(min(num_samples, len(df[df[channel_column] == channel]))))
        print("-" * 80)

print("📍 Checking Unlabelled DataFrame:")
sample_rows_per_channel(unlabelled_df)

print("\n📍 Checking Labelled DataFrame:")
sample_rows_per_channel(labelled_df)


📍 Checking Unlabelled DataFrame:
📌 Sample rows from channel: Da7ee7


,Channel,EpisodeTitle,Original Script,Processed Script,Category
71,Da7ee7,كلوب الدحيح,"""كلوب""، حبيبي،\nاحنا عايزين مصلحتك،\nانت لازم ...",كلوب حبيبي احنا عايزين مصلحتك انت لازم تعتزل و...,
117,Da7ee7,ماركو بولو الدحيح,"ها يا ""ماركو بولو""،\nبقالك سنين بتلفّ حوالين ا...",ها يا ماركو بولو بقالك سنين بتلفّ حوالين العال...,
135,Da7ee7,كوكب القرود الدحيح,ها يا إنسان؟\nعاجبك كدا منظرك؟\nياه!\nحرام علي...,ها يا انسان عاجبك كدا منظرك ياه حرام عليكم يا ...,


--------------------------------------------------------------------------------
📌 Sample rows from channel: Elsaha


,Channel,EpisodeTitle,Original Script,Processed Script,Category
167,Elsaha,توثيق المطبخ المصري.. حكاية سميرة عبدالقادر,احنا 7000 سنه لا مش عايزين شعرات احنا بس\nمترب...,احنا 7000 سنه مش عايزين شعرات احنا بس متربين ع...,
191,Elsaha,البحث عن قاتل أمي.. حكاية نادية مبروك,كان مؤلم قوي ان استرجع شعور امي بعد ما\nضربت ب...,مؤلم قوي ان استرجع شعور امي ضربت بالنار واحد ث...,
176,Elsaha,طب القوات المسلحة.. حكاية ميدالية ذهبية,فكره بتاعتنا اللي انت بتقدم زي كده او\nشريحه ا...,فكره بتاعتنا اللي انت بتقدم زي كده او شريحه ال...,


--------------------------------------------------------------------------------

📍 Checking Labelled DataFrame:
📌 Sample rows from channel: Kefaya Ba2a


...

--------------------------------------------------------------------------------
📌 Sample rows from channel: Fi_Al_Hadaraa


,Channel,Episode Title,Original Script,Processed Script,Dialogue,Type,Length,Category
72,Fi_Al_Hadaraa,صديقي_الإنسان_بتحب_الصدر_ولا_الورك__في_الحضارة,طب ايه\nالليل بيخلص وكاتب التاريخ بيغلق صفحاته...,طب ايه الليل بيخلص وكاتب التاريخ بيغلق صفحاته ...,,Youtube,00:17:09,People & Blogs
94,Fi_Al_Hadaraa,هالاند_تريبل_كابتن__في_الحضارة,يعني خلاص؟\nما فيش منّه تاني؟\nما خلاص بقى يا ...,يعني خلاص فيش منّه تاني خلاص بقي يا نجم كبرنا ...,,Youtube,00:24:18,Entertainment
82,Fi_Al_Hadaraa,متلازمة_المحتال__صديقي_الانسان_هل_تستحق_منصبك_...,"أيوة، حاضر.\nأيوة.\n""ريم""؟!\nإيه يا بنتي اللي ...",ايوه حاضر ايوه ريم ايه يا بنتي اللي انتي عاملا...,,Youtube,00:13:07,People & Blogs


--------------------------------------------------------------------------------
📌 Sample rows from channel: Al_Mokhbir_Al_Eqtisadi


,Channel,Episode Title,Original Script,Processed Script,Dialogue,Type,Length,Category
166,Al_Mokhbir_Al_Eqtisadi,المخبر_الاقتصادي___لماذا_قد_ينهار_اقتصاد_أمريك...,في فبراير 2023\nيعني بعد اربع سنين تقريبا من ا...,فبراير 2023 يعني اربع سنين تقريبا النهارده طبع...,,Youtube,00:14:17,Entertainment
293,Al_Mokhbir_Al_Eqtisadi,المخبر_الاقتصادي_لماذا_ستختفي_كميات_ضخمة_من_ال...,في 31 مارس اللي فات\nالرئيس الصيني شي جين بينغ...,31 مارس اللي فات الرئيس الصيني شي جين بينغ نشر...,,Youtube,00:13:25,Entertainment
263,Al_Mokhbir_Al_Eqtisadi,المخبر_الاقتصادي__كيف_ترفع_الشركات_أسعار_المنت...,في اواخر 2016 حصل هجوم كبير من\nالمستهلكين في ...,اواخر 2016 حصل هجوم كبير المستهلكين بريطانيا ع...,,Youtube,00:15:12,Entertainment


--------------------------------------------------------------------------------


### 5. Data Preparation
	- Category selection? Handle Class Imbalance
	- Feature extraction (bag of words, TF-IDF, word embeddings / vectorization)
	- Encoding
	- Data splitting

In [212]:
!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter

# 1. Category Selection and Class Imbalance Handling
print("Category distribution:")
category_counts = labelled_df['Category'].value_counts()
print(category_counts)

# Handling class imbalance options:
# a. Remove categories with very few samples (optional)
MIN_SAMPLES = 5
valid_categories = category_counts[category_counts >=
                                   MIN_SAMPLES].index.tolist()
filtered_df = labelled_df[labelled_df['Category'].isin(valid_categories)]

print(f"Original dataset size: {len(labelled_df)}")
print(f"Filtered dataset size: {len(filtered_df)}")

# 2. Feature Extraction
# Choose one of the following feature extraction methods:

# Option 1: Bag of Words
count_vectorizer = CountVectorizer(min_df=2, max_df=0.95)
X_bow = count_vectorizer.fit_transform(filtered_df['Processed Script'])

# Option 2: TF-IDF (usually performs better for text classification)
tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=0.95)
X_tfidf = tfidf_vectorizer.fit_transform(filtered_df['Processed Script'])

# Let's use TF-IDF for our primary feature set
X = X_tfidf

# 3. Encoding the target variable (categories)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(filtered_df['Category'])

# Display the encoding mapping
category_mapping = dict(
    zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("\nCategory encoding mapping:")
for category, code in category_mapping.items():
    print(f"{category} -> {code}")

# 4. Data Splitting
# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Further handle class imbalance with SMOTE (only on training data)
print("\nClass distribution before SMOTE:")
print(Counter(y_train))

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("\nClass distribution after SMOTE:")
print(Counter(y_train_resampled))

# 6. Prepare a function to process and vectorize unlabelled data
def prepare_unlabelled_data(unlabelled_df, vectorizer):
    """Process and vectorize unlabelled data using the same vectorizer as the training data"""
    X_unlabelled = vectorizer.transform(unlabelled_df['Processed Script'])
    return X_unlabelled


# Process unlabelled data using the same vectorizer
X_unlabelled = prepare_unlabelled_data(unlabelled_df, tfidf_vectorizer)

print(f"\nTraining set shape: {X_train_resampled.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"Unlabelled data shape: {X_unlabelled.shape}")

Category distribution:
Category
Entertainment     229
People & Blogs     72
Education          24
Comedy              4
                    2
Name: count, dtype: int64
Original dataset size: 331
Filtered dataset size: 325

Category encoding mapping:
Education -> 0
Entertainment -> 1
People & Blogs -> 2

Class distribution before SMOTE:
Counter({1: 183, 2: 58, 0: 19})

Class distribution after SMOTE:
Counter({1: 183, 2: 183, 0: 183})


/Users/sherifahammoud/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Training set shape: (549, 34982)
Testing set shape: (65, 34982)
Unlabelled data shape: (193, 34982)
